In [6]:
import os
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/컴퓨터 비전/computer-vision

Mounted at /content/drive/
/content/drive/MyDrive/컴퓨터 비전/computer-vision


In [15]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from dataloader.dataset import CustomDataset
from model.unet import UNet
from tool.encode import rle_encode
import albumentations as A
from PIL import Image
from albumentations.pytorch import ToTensorV2

def set_dataloader():
    def set_tta():
        transform = A.Compose(
          [
              A.Resize(224, 224),
              A.Normalize(),
              ToTensorV2()
          ])
        return transform
    test_dataset = CustomDataset(csv_file='./test.csv', transform=set_tta(), infer=True)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)
    return test_dataloader

def set_model():
    model = UNet()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    return model, device

def inference():
  model, device = set_model()
  test_dataloader = set_dataloader()

  with torch.no_grad():
      model.eval()
      result = []
      for images in tqdm(test_dataloader):
          images = images.float().to(device)
          outputs = model(images)
          outputs = torch.softmax(outputs, dim=1).cpu()
          outputs = torch.argmax(outputs, dim=1).numpy()
          # batch에 존재하는 각 이미지에 대해서 반복
          for pred in outputs:
              pred = pred.astype(np.uint8)
              pred = Image.fromarray(pred) # 이미지로 변환
              pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
              pred = np.array(pred) # 다시 수치로 변환
              # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
              for class_id in range(12):
                  class_mask = (pred == class_id).astype(np.uint8)
                  if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
                      mask_rle = rle_encode(class_mask)
                      result.append(mask_rle)
                  else: # 마스크가 존재하지 않는 경우 -1
                      result.append(-1)
  submit = pd.read_csv('./sample_submission.csv')
  submit['mask_rle'] = result
  submit.to_csv('./baseline_submit.csv', index=False)